In [2]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import torch
import evaluate
import mlflow
import numpy as np

from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)

In [4]:
mlflow.set_tracking_uri("http://mlflow:5000")
tracking_uri = mlflow.get_tracking_uri()
print("Current tracking uri: {}".format(tracking_uri))

os.environ["MLFLOW_EXPERIMENT_NAME"] = "NER"
# os.environ["MLFLOW_FLATTEN_PARAMS"] = "True"

# os.environ["MLFLOW_TRACKING_URI"] = tracking_uri

Current tracking uri: http://mlflow:5000


In [5]:
raw_dataset = load_dataset("kosta-naumenko/medflex", split='train', download_mode='force_redownload', verification_mode='no_checks')
raw_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/394 [00:00<?, ? examples/s]

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 394
})

In [20]:
model_name = "alexyalunin/RuBioRoBERTa"
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

In [7]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [8]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, 
        max_length=512, padding=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [9]:
tokenized_dataset = raw_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_dataset.column_names,
)

Map:   0%|          | 0/394 [00:00<?, ? examples/s]

In [10]:
id = 4
input_ids, attention_mask, labels = list(tokenized_dataset[id].values())
for i in range(len(input_ids)):
    if labels[i] > 0:
        if labels[i] == 1:
            print(" ")
        print(tokenizer.decode(input_ids[i]), end='')

 
 гликемия 12-13 ммоль/л, 
 ИМТ 26,53 кг/м 2 
 сухость во рту, 
 жажды, 
 учащённого мочеиспускания. 
 гликемия при контроле 1 раз в день натощак 12-13 ммоль/л,

In [11]:
seqeval = evaluate.load("seqeval")
label_list = ['O', 'B', 'I']


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [12]:
id2label = {
    0: "O",
    1: "B",
    2: "I",
}
label2id = {
    "O": 0,
    "B": 1,
    "I": 2,
}

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
eval_dataset = tokenized_dataset.train_test_split(test_size=0.1)['test']

In [22]:
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    device_map={'': torch.cuda.current_device()},
    cache_dir='.cache',
    num_labels=3,
    id2label=id2label,
    label2id=label2id
    )

for param in model.roberta.parameters():
    param.requires_grad = False

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at alexyalunin/RuBioRoBERTa and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
cur_run_id = 1
num_train_epochs=100

name = "RuBioRoBERTa-finetune-head"
run_name = f'{name}-{cur_run_id:02}'
output_dir = f'./logs/{run_name}'

training_args = TrainingArguments(
    output_dir=output_dir,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_steps=1,
    save_strategy='epoch',
    load_best_model_at_end=True,
    report_to="mlflow",
    run_name=run_name
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
mlflow.end_run()

/home/knaumenko/.conda/envs/NER/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.645400,0.607616,0.000000,0.000000,0.000000,0.768536
2,0.623800,0.575525,0.000000,0.000000,0.000000,0.776257
3,0.501400,0.555004,0.000000,0.000000,0.000000,0.779065
4,0.656100,0.537695,0.000000,0.000000,0.000000,0.784154
5,0.534800,0.526379,0.000000,0.000000,0.000000,0.787488
6,0.470200,0.518981,0.000000,0.000000,0.000000,0.786435
7,0.548600,0.511208,0.000000,0.000000,0.000000,0.790032
8,0.563400,0.508320,0.000000,0.000000,0.000000,0.790471
9,0.532400,0.503448,0.000000,0.000000,0.000000,0.792928
10,0.509400,0.498862,0.002817,0.004115,0.003344,0.794858


In [ ]:
path_to_model = f'../../models/{run_name}.pt'
torch.save(model, path_to_model)

In [45]:
model2 = torch.load('../../models/rubio_frozen.pt')

In [46]:
input_ids = torch.LongTensor(tokenized_dataset['input_ids']).to(model.device)
attention_mask = torch.LongTensor(tokenized_dataset['attention_mask']).to(model.device)

preds = model2(input_ids=input_ids, attention_mask=attention_mask)
p = [preds['logits'].detach().cpu(), tokenized_dataset['labels']]
compute_metrics(p)

{'precision': 0.0017035775127768314,
 'recall': 0.003173164097914778,
 'f1': 0.002216943784639747,
 'accuracy': 0.7840474930935167}

In [26]:
input_ids = torch.LongTensor(tokenized_dataset['input_ids']).to(model.device)
attention_mask = torch.LongTensor(tokenized_dataset['attention_mask']).to(model.device)

preds = model(input_ids=input_ids, attention_mask=attention_mask)
p = [preds['logits'].detach().cpu(), tokenized_dataset['labels']]
compute_metrics(p)

{'precision': 0.013938594838952722,
 'recall': 0.033544877606527655,
 'f1': 0.019693945442448436,
 'accuracy': 0.8131428907306177}

In [33]:
logits = preds.logits.argmax(axis=2).detach().cpu()

In [38]:
(logits == 1).sum()

tensor(2697)